<a href="https://colab.research.google.com/github/v5v5/trading/blob/main/trade_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

URL to download quotes EURUSD

http://export.finam.ru/EURUSD_210101_210425.csv?market=1&em=175924&code=EURUSD&apply=0&df=1&mf=1&yf=2021&from=01.01.2021&dt=25&mt=4&yt=2021&to=25.04.2021&p=8&f=EURUSD_210101_210425&e=.csv&cn=EURUSD&dtf=1&tmf=1&MSOR=0&mstime=on&mstimever=1&sep=3&sep2=1&datf=1&at=1

In [28]:
import pandas as pd
from urllib.request import Request, urlopen
from io import StringIO

In [29]:
# url = 'http://export.finam.ru/EURUSD_210101_210425.csv?market=1&em=175924&code=EURUSD&apply=0&df=1&mf=1&yf=2021&from=01.01.2021&dt=25&mt=4&yt=2021&to=25.04.2021&p=8&f=EURUSD_210101_210425&e=.csv&cn=EURUSD&dtf=1&tmf=1&MSOR=0&mstime=on&mstimever=1&sep=3&sep2=1&datf=1&at=1'
# url = 'http://export.finam.ru/EURUSD_150101_210425.csv?market=1&em=175924&code=EURUSD&apply=0&df=1&mf=1&yf=2015&from=01.01.2015&dt=25&mt=4&yt=2021&to=25.04.2021&p=8&f=EURUSD_150101_210425&e=.csv&cn=EURUSD&dtf=1&tmf=1&MSOR=0&mstime=on&mstimever=1&sep=3&sep2=1&datf=1&at=1'
url = 'http://export.finam.ru/EURUSD_000101_210425.csv' + \
'?market=1&em=175924&code=EURUSD&apply=0' + \
'&df=1&mf=1&yf=2000&from=01.01.2000&dt=25&mt=4&yt=2021&to=25.04.2021' + \
'&p=8&f=EURUSD_000101_210425&e=.csv&cn=EURUSD' + \
'&dtf=1&tmf=1' + \
'&MSOR=0&mstime=on&mstimever=1' + \
'&sep=3&sep2=1&datf=1&at=1'

In [30]:
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
page = urlopen(req)
content = page.read()

In [31]:
text = content.decode('UTF-8')

In [32]:
print(text[:300])

<TICKER>;<PER>;<DATE>;<TIME>;<OPEN>;<HIGH>;<LOW>;<CLOSE>;<VOL>
EURUSD;D;20130620;000000;292.6000000;300.0000000;244.5500000;278.2300000;4180
EURUSD;D;20130621;000000;277.6000000;279.9000000;260.5000000;262.3800000;4930
EURUSD;D;20130624;000000;265.8600000;265.8600000;254.0000000;254.5000000;6658


In [33]:
data = StringIO(text)

In [34]:
df1 = pd.read_csv(data, sep=';', header='infer')

In [35]:
df1.head()

,<TICKER>,<PER>,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,EURUSD,D,20130620,0,292.60,300.00,244.55,278.23,4180
1,EURUSD,D,20130621,0,277.60,279.90,260.50,262.38,4930
2,EURUSD,D,20130624,0,265.86,265.86,254.00,254.50,6658
3,EURUSD,D,20130625,0,251.19,255.94,244.95,252.17,5569
4,EURUSD,D,20130626,0,247.32,248.51,236.43,237.07,11800


In [36]:
df1['<DATE-FORMATED>'] = pd.to_datetime(df1['<DATE>'], format='%Y%m%d')

In [37]:
df1.head()

,<TICKER>,<PER>,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>,<DATE-FORMATED>
0,EURUSD,D,20130620,0,292.60,300.00,244.55,278.23,4180,2013-06-20
1,EURUSD,D,20130621,0,277.60,279.90,260.50,262.38,4930,2013-06-21
2,EURUSD,D,20130624,0,265.86,265.86,254.00,254.50,6658,2013-06-24
3,EURUSD,D,20130625,0,251.19,255.94,244.95,252.17,5569,2013-06-25
4,EURUSD,D,20130626,0,247.32,248.51,236.43,237.07,11800,2013-06-26


In [38]:
df1['<DAY-OF-WEEK>'] = df1['<DATE-FORMATED>'].dt.dayofweek

In [39]:
df1.head()

,<TICKER>,<PER>,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>,<DATE-FORMATED>,<DAY-OF-WEEK>
0,EURUSD,D,20130620,0,292.60,300.00,244.55,278.23,4180,2013-06-20,3
1,EURUSD,D,20130621,0,277.60,279.90,260.50,262.38,4930,2013-06-21,4
2,EURUSD,D,20130624,0,265.86,265.86,254.00,254.50,6658,2013-06-24,0
3,EURUSD,D,20130625,0,251.19,255.94,244.95,252.17,5569,2013-06-25,1
4,EURUSD,D,20130626,0,247.32,248.51,236.43,237.07,11800,2013-06-26,2


In [47]:
df2 = df1.drop(['<TICKER>','<PER>','<DATE>','<TIME>','<HIGH>', '<LOW>', '<VOL>'], axis=1)

In [48]:
df2.head()

,<OPEN>,<CLOSE>,<DATE-FORMATED>,<DAY-OF-WEEK>
0,292.60,278.23,2013-06-20,3
1,277.60,262.38,2013-06-21,4
2,265.86,254.50,2013-06-24,0
3,251.19,252.17,2013-06-25,1
4,247.32,237.07,2013-06-26,2
